__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
# avito_path = 'sevastopol/kvartiry/prodam'
avito_path = 'sevastopol/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-08 16:01:33 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-08 16:01:33 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-08 16:01:36 | AdsListParser: downloader init
[INFO    ] 2022-09-08 16:01:36 | AvitoParser: init
[INFO    ] 2022-09-08 16:02:42 | AvitoParser: 23 pages for read
[INFO    ] 2022-09-08 16:02:42 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-08 16:02:45 | AdsListParser: read page 1
[INFO    ] 2022-09-08 16:02:47 | AdsListParser: read page 2
[INFO    ] 2022-09-08 16:03:47 | AdsListParser: read page 3
[INFO    ] 2022-09-08 16:03:49 | AdsListParser: read page 4
[INFO    ] 2022-09-08 16:03:53 | AdsListParser: read page 5
[INFO    ] 2022-09-08 16:03:55 | AdsListParser: read page 6
[INFO    ] 2022-09-08 16:03:59 | AdsListParser: read page 7
[INFO    ] 2022-09-08 16:04:01 | AdsListParser: read page 8
[INFO    ] 2022-09-08 16:04:03 | AdsListParser: read page 9
[INFO    ] 2022-09-08 16:04:07 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1086
1077


In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [8]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
748,2266929716,Объявление «Участок 10 сот. (ИЖС)» 25 фотографий,7000000,,"ул. Разведчика Колобуна, 7","Собственник продает земельный участок по адресу: г. Севастополь, ул. Разведчика Колобуна,7 /кадастровый номер 91:04:002013:5/, расположенный в видовом месте Тсн «Опера», возвышенность между Делегардовой и Хомутовой балкой. \nВ данный момент для коммерческог",17,2022-09-08 16:04:40.227241
1044,2411382491,Объявление «Участок 4 сот. (ИЖС)» 3 фотографии,3500000,,"садоводческое товарищество Первомайское, Первомайская ул.","Продаю свой участок на Фиоленте, без посредников, ст первомайское. Расположен в историческом месте по дороге к старинному монастырю и живописному парку. Участок просто чудный: ровный, с хорошими подъездными путями, 5 минут до моря, в шаговой доступности об",23,2022-09-08 16:04:40.227241
